In [2]:
import pandas as pd
import matplotlib.pyplot as plt

renewables_csv = 'renewable-share-energy.csv'
co2_csv = 'consumption-co2-per-capita.csv'
gdp_file = "world_bank_GDP_per_country.csv"


In [3]:
def renewables_gdp_percapita(directory, country_code):
    dataframe = pd.read_csv(directory, skiprows=range(4))
    
    # Filter the DataFrame based on the regional entity (Country Code)
    filtered_data = dataframe[dataframe['Country Code'] == country_code]
    
    # Extract the years and corresponding GDP values into a Pandas DataFrame
    gdp_data_df = filtered_data.iloc[:, 4:]  # Assuming GDP values start from the 5th column
    gdp_data_df = gdp_data_df.T.reset_index()
    gdp_data_df.columns = ['Year', 'GDP (current US$)']
    
    # Convert 'Year' column to numeric and exclude non-numeric values
    gdp_data_df['Year'] = pd.to_numeric(gdp_data_df['Year'], errors='coerce')
    gdp_data_df = gdp_data_df.dropna(subset=['Year'])
    
    return gdp_data_df

In [4]:
renewables_gdp_percapita(gdp_file, "RUS")

,Year,GDP (current US$)
0,1960.0,NaN
1,1961.0,NaN
2,1962.0,NaN
3,1963.0,NaN
4,1964.0,NaN
...,...,...
58,2018.0,1.657329e+12
59,2019.0,1.693115e+12
60,2020.0,1.493076e+12
61,2021.0,1.836892e+12


In [11]:
import pandas as pd

def vertical_to_horizontal(csv_file):
    # Read the CSV file into a DataFrame
    dataframe = pd.read_csv(csv_file)

    # Pivot the DataFrame based on 'Entity' column
    pivoted_df = dataframe.pivot_table(index='Year', columns='Entity', values=dataframe.columns[3]).reset_index()

    return pivoted_df

# flipping the data and removing unused years.

renewables_flipped = vertical_to_horizontal(renewables_csv)
renewables_flipped = renewables_flipped[(renewables_flipped['Year'] >= 1990) & (renewables_flipped['Year'] <= 2020)]

co2_flipped = vertical_to_horizontal(co2_csv)
co2_flipped = co2_flipped[(co2_flipped['Year'] >= 1990) & (co2_flipped['Year'] <= 2020)]

non_country_entities = [
    'High-income countries', 'South America', 'Europe', 'Oceania', 'Africa',
    'Lower-middle-income countries', 'Upper-middle-income countries', 'World',
    'North America', 'European Union (27)', 'Asia'
]

def filter_countries_1990_2020(df):
    # Filter the DataFrame to keep only rows for 1990 and 2020
    df_1990 = df[df['Year'] == 1990]
    df_2020 = df[df['Year'] == 2020]

    # Extract the countries that have data in both 1990 and 2020
    countries_1990 = set(df_1990.columns[df_1990.notnull().any()])
    countries_2020 = set(df_2020.columns[df_2020.notnull().any()])

    countries_both_years = list(countries_1990.intersection(countries_2020))

    # Filter the original DataFrame to include only columns (countries) present in both 1990 and 2020
    filtered_df = df[df.columns[df.columns.isin(countries_both_years)]]

    return filtered_df

renewables_flipped = filter_countries_1990_2020(renewables_flipped)
co2_flipped = filter_countries_1990_2020(co2_flipped)

def filter_common_countries(df1, df2):
    countries_df1 = set(df1.columns)
    countries_df2 = set(df2.columns)
    common_countries = list(countries_df1.intersection(countries_df2))

    # Exclude 'Year' from removal if present in common columns
    if 'Year' in common_countries:
        common_countries.remove('Year')

    # Filter DataFrames to include only common countries (excluding 'Year')
    filtered_df1 = df1[['Year'] + common_countries]
    filtered_df2 = df2[['Year'] + common_countries]

    return filtered_df1, filtered_df2

renewables_filtered, co2_filtered = filter_common_countries(renewables_flipped, co2_flipped)

# Remove non-country entities from the DataFrames
renewables_filtered = renewables_filtered.drop(columns=non_country_entities, errors='ignore')
co2_filtered = co2_filtered.drop(columns=non_country_entities, errors='ignore')


display(renewables_filtered)
display(co2_filtered)


Entity,Year,Lithuania,Estonia,Morocco,Azerbaijan,South Korea,Bangladesh,Russia,Slovakia,Austria,...,Kazakhstan,Venezuela,Greece,Colombia,Belarus,Croatia,United States,Italy,Australia,Ukraine
25,1990,0.606307,0.000000,4.359246,1.881660,1.299518,3.539271,4.898305,2.238396,28.908712,...,2.534389,18.119040,1.822694,32.548810,0.012692,11.523735,4.724676,5.543998,4.459392,0.973168
26,1991,0.468239,0.000000,4.325900,2.077973,0.855261,3.477609,5.003191,1.873833,27.587229,...,2.479837,21.675652,3.145859,31.965420,0.011449,19.537254,4.724286,7.020093,4.740063,1.156305
27,1992,0.703257,0.000000,3.039623,2.409933,0.674040,3.028885,5.336780,2.679717,31.037298,...,2.395667,21.255587,2.205222,24.941273,0.012469,15.994586,4.261231,7.082338,4.731717,0.921140
28,1993,1.136311,0.000000,1.382297,3.857215,0.838968,2.104329,5.780297,4.882404,32.088062,...,3.015272,21.576940,2.292617,28.616268,0.016666,16.087374,4.571410,7.147924,4.759429,1.471511
29,1994,1.514459,0.000000,2.377947,3.167003,0.431987,2.761680,6.394297,6.225819,31.296307,...,4.072831,21.469650,2.551279,30.409860,0.019936,18.960016,4.254817,7.645524,4.527182,1.890063
30,1995,1.118191,0.030810,1.763483,2.864073,0.510264,1.026995,6.769154,6.879693,31.515926,...,4.114585,20.915274,3.326749,29.690346,0.023589,19.160814,4.762141,6.312267,4.239725,1.591071
31,1996,0.897941,0.024179,5.402516,3.449877,0.436198,1.947564,6.157421,5.918110,28.951240,...,4.159171,22.462038,3.993021,31.470236,0.018399,24.701183,5.018571,7.041860,4.236591,1.480491
32,1997,0.881222,0.039272,5.531386,4.012825,0.402687,1.803428,6.642448,5.796413,29.963510,...,4.181545,22.598196,3.468253,27.651566,0.023874,18.543142,5.115221,6.921144,4.145840,1.752618
33,1998,1.161542,0.065671,4.600728,4.555314,0.659769,2.037124,6.699800,6.022059,30.441757,...,4.193706,21.551676,3.156418,26.757204,0.032552,18.063107,4.695756,6.806154,4.037756,2.900869
34,1999,1.394003,0.099236,2.058802,3.524421,0.593689,1.939729,6.699299,6.261234,32.248478,...,4.455529,22.529486,3.964061,31.315977,0.022708,21.245943,4.583452,7.312841,4.008933,2.680325


Entity,Year,Lithuania,Estonia,Morocco,Azerbaijan,South Korea,Bangladesh,Russia,Slovakia,Austria,...,Kazakhstan,Venezuela,Greece,Colombia,Belarus,Croatia,United States,Italy,Australia,Ukraine
0,1990,11.455255,19.982485,1.240285,3.757798,7.257062,0.170461,13.794909,13.914664,11.629516,...,17.148327,4.762998,9.354046,1.900071,5.879944,4.303235,20.347502,9.988994,14.263723,9.703509
1,1991,12.141571,19.354488,1.231995,2.813551,7.683075,0.177141,13.999880,10.875343,11.735809,...,12.502390,4.826739,9.171034,1.816593,6.373846,3.223149,19.725270,9.887093,14.118852,8.813906
2,1992,13.901977,15.631665,1.409892,1.688845,8.837342,0.185828,5.236676,10.595663,11.870350,...,7.600424,4.589158,9.975104,1.982240,4.293134,3.786203,20.175217,10.771868,14.113060,8.487364
3,1993,8.606816,11.952082,1.359150,3.077114,8.983179,0.185000,7.485521,9.722796,11.266796,...,10.002325,5.136385,9.400922,1.995550,3.615274,3.793324,20.126003,9.839956,14.031878,7.187042
4,1994,7.652970,11.291018,1.359539,2.395709,9.412948,0.200583,7.909903,8.417370,11.180918,...,8.810326,5.035499,9.311268,2.056611,2.861869,3.689194,20.247800,9.532290,14.365695,5.299610
5,1995,7.510003,10.810749,1.368468,2.931235,9.938135,0.223746,7.871611,8.716562,11.673425,...,8.170543,5.257839,9.353244,1.844342,3.317751,4.082282,20.217650,9.890141,14.520001,4.263806
6,1996,6.154727,11.686723,1.235440,3.552704,10.738061,0.226845,8.465579,9.014393,12.658384,...,7.382071,4.386655,9.128911,1.809466,4.589332,4.124131,20.590729,9.300487,14.703377,5.650606
7,1997,5.894873,11.224070,1.171168,3.194477,10.822609,0.238152,7.967262,8.813198,11.905045,...,7.060441,5.163228,9.190524,1.918751,4.326876,4.420623,20.717155,9.382641,14.954647,5.491249
8,1998,6.070131,12.119434,1.252733,3.738707,7.779057,0.232896,7.150222,9.033363,12.322413,...,7.486028,6.608773,9.795411,1.894471,6.867243,4.388584,21.054504,9.564389,15.831035,4.883518
9,1999,5.478316,14.091313,1.345823,3.291646,9.319485,0.237875,6.861424,8.426398,12.082646,...,6.598050,4.856058,9.391486,1.458585,6.574572,4.778892,21.440773,9.813859,15.713442,4.802996
